# 项目地址  
https://python.langchain.com/docs/tutorials/llm_chain/

通过这个项目，你将获得的能力：  
1.使用大语言模型  
2.使用提示词模版  
3.使用LangSmith来调试和追踪你的应用问题  

# 代码环境设置
Jupyter Notebook or Jupyter Lab
自行找文档下载安装哈

## 安装langchain

In [ ]:
pip install langchain

: 

In [ ]:
import langchain

: 

# 设置Langsmith
1. LangSmith是做什么的  
    - 跟踪调试：记录模型运行过程，定位问题。
    - 测试评估：批量测试模型表现，支持自定义标准。
    - 实时监控：跟踪生产环境性能和用户交互。
    - Prompt 管理：优化和管理提示词，支持版本控制。  
    
2. Address：
    https://smith.langchain.com/onboarding?organizationId=d33de966-2acd-4e99-884a-4a51a8600109&step=1
3. 其他:
    - 需要注册
    - 需要生成API key用来交互，服务在云端
    - 如没有监控需求，也可以选择不启用这个监控服务，对后续无影响

In [ ]:
# 代码

import getpass
import os  

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()  

: 

**代码解释**
1. getpass：getpass 是一个用于安全输入密码或敏感信息的工具。它的主要功能是让用户在终端输入内容时不回显（即输入时不会显示字符，通常用于密码输入）。
2. os：os 模块提供了与操作系统交互的功能，包括文件操作、路径处理和环境变量管理等。
3. os.environ["LANGSMITH_TRACING"] = "true" 用来激活LangSmith服务，在后续的Langchain模块中会检测这个环境变量是否启用，如果启用会和LangSmith服务进行交互进而开启监控。
4. os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
    1. 用来拿到LangSmith的API Key
    2. getpass.getpass() 会提示用户在终端输入内容（默认提示是 "Password: "），输入的内容不会显示在屏幕上
    3. Appkey需要注册账号，然后在网站内生成即可，也可以选择不启用这个监控服务，跳过即可。

## 使用大语言模型

**安装langchain的集成**  
pip install -qU "langchain-deepseek"

- 安装Deepseek的Langchain扩展，用于和Deepseek接口进行交互。
- Langchain也可以使用其他的模型，具体请直接查询官方文档代码，集成会有不一样，代码会有所变化。

In [ ]:
# 配置接口参数，创建语言模型的实例
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-chat",  # 指定使用的模型
    temperature=0,  # 控制生成文本的随机性
    max_tokens=None,  # 生成的最大 token 数限制
    timeout=None,  # 请求超时时间
    max_retries=2, # 请求失败时的最大重试次数
    # other params...
)

: 

In [ ]:
# 前置需要进入deepseek官网注册并生成API_KEY，记得保存好，否则丢失后需要重新生成。
# 这一行检查程序是否已经有 Deepseek API 密钥。如果没有，就需要用户手动输入。
if not os.getenv("DEEPSEEK_API_KEY"):
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter your DeepSeek API key: ")

: 

**代码解释**
- os.getenv(key) 是 os 模块的一个函数，用于读取环境变量的值。
- 如果环境变量 DEEPSEEK_API_KEY 已设置（比如通过终端的 export DEEPSEEK_API_KEY="your-key"），它返回对应的值（字符串）。
- 如果未设置或值为空，则返回 None。


## 流式处理stream

In [10]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg.content

"J'adore la programmation."

**代码解释**
- 定义一个列表 messages，包含两条消息，每条消息是一个元组 (角色, 内容)。
- 调用语言模型（llm）的 invoke 方法，将 messages 作为输入，生成响应，并将结果赋值给 ai_msg。
- 访问 ai_msg 对象的 content 属性，获取模型生成的文本内容。

In [12]:
for token in llm.stream(messages):
    print(token.content, end="|")

|J|'ad|ore| programmer|.||

**代码解释**
- 通过 llm.stream(messages) 调用语言模型的流式输出方法，并用 for 循环迭代处理返回的每个 token。
- stream 是语言模型提供的一个方法，用于以流式（streaming）方式生成输出。
- 与一次性返回完整结果的 invoke 不同，stream 返回一个迭代器（iterator）或生成器（generator），逐步生成并返回模型输出的每个 token。

## 提示词模版

In [13]:
# 定义一个提示模板，用于将用户输入的英文文本翻译成指定语言并传递给大模型。
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

**代码解释**
- ChatPromptTemplate 是 LangChain 提供的一个工具，用于构建结构化的对话提示模板。
- 它特别适合与支持聊天格式的语言模型（如 ChatDeepSeek、ChatOpenAI）配合使用，允许定义多角色消息（如系统消息、用户消息）。
- prompt_template 是一个可重用的模板对象，允许动态填入 {language} 和 {text} 的值，生成完整的提示。

In [14]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

**代码解释**
- invoke 是 ChatPromptTemplate 的方法，用于将占位符替换为具体值，生成一个具体的提示对象。
- 输入是一个字典，键（language 和 text）对应模板中的占位符，值（"Italian" 和 "hi!"）是替换的内容。

In [15]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

**代码解释**
- 调用 prompt 对象的 to_messages() 方法，返回一个消息列表。

In [22]:
#与大模型交互，将提示词模版传输给大模型，并得到答案
response = llm.invoke(prompt)
print(response.content)

Ciao!
